In [1]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
import psycopg2
from config import db_password
import time

In [7]:
def ETL(state):
    zillow_df = pd.read_csv('Resources/Zillow_Sale_Prices_Zip.csv')
    zillow_df.set_index('RegionName', inplace=True)
    state_df = zillow_df.loc[zillow_df.StateName==state,:]
    state_df = state_df.dropna()
    median09 = state_df.loc[:,'2009-01':'2009-12'].median(axis = 1, skipna = True)
    median19 = state_df.loc[:,'2019-01':'2019-12'].median(axis = 1, skipna = True)
    new_df=pd.concat([median09, median19], axis=1)
    new_name= ['Median Price 2009','Median Price 2019']
    new_df.columns=new_name
    new_df['Inflated Median Price 2009'] = 1.1917 * new_df['Median Price 2009']
    new_df['Percent Change'] = (new_df['Median Price 2019'] - new_df['Inflated Median Price 2009'])/new_df['Inflated Median Price 2009']
    new_df = new_df[['Median Price 2009','Inflated Median Price 2009','Median Price 2019','Percent Change']]
    new_df['Median Price 2009'] = new_df['Median Price 2009'].map("${:,.2f}".format)
    new_df['Inflated Median Price 2009'] = new_df['Inflated Median Price 2009'].map("${:,.2f}".format)
    new_df['Median Price 2019'] = new_df['Median Price 2019'].map("${:,.2f}".format)
    new_df['Percent Change'] = new_df['Percent Change'].map("{:.2%}".format)
    
    return new_df.head()

In [8]:
ETL('Arizona')

,Median Price 2009,Inflated Median Price 2009,Median Price 2019,Percent Change
RegionName,,,,
85022,141900.0,169102.230,237800.0,0.406250
85023,112100.0,133589.570,239150.0,0.790185
85024,170700.0,203423.190,265800.0,0.306636
85027,97950.0,116727.015,213450.0,0.828626
85032,122950.0,146519.515,238200.0,0.625722
